# Overview

**Oftentimes, we want to test some LLMs that are not widely available or are expensive to host. There are various ways to still get your hands dirty with them.**


**In this notebook, I'm showing how to use LangChain's CustomLLM together with <a href="https://openrouter.ai/">OpenRouter.ai</a>'s collection of LLMs (like gpt-4-32k, claude-2, and many more) to use these models in your LangChain apps.**

### <font color='gray'>Loading libraries and variables</font>

In [3]:
import os
import requests, json
from dotenv import load_dotenv
import re
import zipfile
import io
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
load_dotenv()

True

In [18]:
API_KEY = os.getenv('OPENROUTER_API_KEY')

MODEL_CODELLAMA = "phind/phind-codellama-34b"
MODEL_LLAMA3 = "meta-llama/llama-3-70b-instruct"
MODEL_MIXTRAL = "mistralai/mixtral-8x22b-instruct"

# Setting up LLM

In [19]:
# Initliaze models
llm_llama3 = ChatOpenAI(openai_api_key=API_KEY, openai_api_base="https://openrouter.ai/api/v1", model_name=MODEL_LLAMA3)
llm_codellama = ChatOpenAI(openai_api_key=API_KEY, openai_api_base="https://openrouter.ai/api/v1", model_name=MODEL_CODELLAMA)
llm_mixtral = ChatOpenAI(openai_api_key=API_KEY, openai_api_base="https://openrouter.ai/api/v1", model_name=MODEL_MIXTRAL)

In [20]:
def get_zip_structure(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_structure = io.StringIO()
        zip_ref.printdir(file=zip_structure)
        structure_text = zip_structure.getvalue().strip()
    return structure_text

In [21]:
INSTRUCT_WRAP_CODE = "\nWrap your code in ```python\n{CODE}\n```"

In [68]:
input = "In this PDF file, replace all occurrences of 'hi' with 'hello' in place without extracting text"

# input = "Convert the images in this zip file to a pdf by adding all images contained within to the PDF. Make sure to pay attention to the structure inside the zip file to make sure all code correctly handles the zipfile."

file = "resume.pdf"

# Generate detailed prompt from simple user prompt
template_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful prompting assistant.\
    You must write instructions for creating a python script for completing the below user prompt.\
    Your job is to expand on the instructions with a plan and defined steps to completion as well as any supplemental information. """),
    ("user", "User Prompt: {input}\nFile: {file}\n")
])

template_code = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful coding assistant. Write code based on the instructions with redundancy for errors. Also write a requirements.txt wrapped in ```requirements\n``` with the specific packages and version numbers."),
    ("user", "Instructions: {input}\nFile: {file}")
])

# file_contents = "\nContents of unzipped file:\n" + get_zip_structure(file)


def gen_content(llm_prompt, llm_code):
    prompt_chain = template_prompt | llm_prompt | output_parser
    
    code_chain = template_code | llm_code | output_parser
    
    prompt_output = prompt_chain.invoke({"input": input, "file": file})
    
    prompt_output += INSTRUCT_WRAP_CODE
    
    code_output = code_chain.invoke({"input": prompt_output, "file": file})
    
    python_code = re.search(r'```python(?s:(.*?))```', code_output).group(1)
    
    print(python_code)

    return code_output

# gen_content(llm_mixtral, llm_mixtral)
output = gen_content(llm_mixtral, llm_llama3)
# gen_content(llm_wizard22, llm_wizard22)


try:
    from PyPDF2 import PdfFileReader, PdfFileWriter
except ImportError:
    print("Error: PyPDF2 is not installed. Please install it using pip: `pip install PyPDF2`")
    exit(1)

try:
    pdf_file = PdfFileReader("resume.pdf")
except FileNotFoundError:
    print("Error: File 'resume.pdf' not found. Please ensure the file exists in the same directory.")
    exit(1)

pdf_writer = PdfFileWriter()

try:
    for page in range(pdf_file.getNumPages()):
        pdf_page = pdf_file.getPage(page)
        pdf_page.mergeTranslatedPage(pdf_page.getContents().replace(b'hi', b'hello'), 0, 0)
        pdf_writer.addPage(pdf_page)
except Exception as e:
    print(f"Error: An error occurred while processing the PDF file: {e}")
    exit(1)

try:
    with open('new_resume.pdf', 'wb') as fh:
        pdf_writer.write(fh)
except Exception as e:
    print(f"Error: An error occurred while writing the new PDF file: {e}")
    exit(1)

print("Successfully replaced 'hi' with 'hello' in the PDF file.")



In [69]:
print(output)

Here is the code with redundancy for errors and a requirements.txt file:

**Code:**
```python
try:
    from PyPDF2 import PdfFileReader, PdfFileWriter
except ImportError:
    print("Error: PyPDF2 is not installed. Please install it using pip: `pip install PyPDF2`")
    exit(1)

try:
    pdf_file = PdfFileReader("resume.pdf")
except FileNotFoundError:
    print("Error: File 'resume.pdf' not found. Please ensure the file exists in the same directory.")
    exit(1)

pdf_writer = PdfFileWriter()

try:
    for page in range(pdf_file.getNumPages()):
        pdf_page = pdf_file.getPage(page)
        pdf_page.mergeTranslatedPage(pdf_page.getContents().replace(b'hi', b'hello'), 0, 0)
        pdf_writer.addPage(pdf_page)
except Exception as e:
    print(f"Error: An error occurred while processing the PDF file: {e}")
    exit(1)

try:
    with open('new_resume.pdf', 'wb') as fh:
        pdf_writer.write(fh)
except Exception as e:
    print(f"Error: An error occurred while writing the new PDF file

In [1]:
!pip install PyPDF2==1.26.0

DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tree_of_thoughts-0.1.8-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
try:
    from PyPDF2 import PdfFileReader, PdfFileWriter
except ImportError:
    print("Error: PyPDF2 is not installed. Please install it using pip: `pip install PyPDF2`")
    exit(1)

try:
    pdf_file = PdfFileReader("resume.pdf")
except FileNotFoundError:
    print("Error: File 'resume.pdf' not found. Please ensure the file exists in the same directory.")
    exit(1)

pdf_writer = PdfFileWriter()

try:
    for page in range(pdf_file.getNumPages()):
        pdf_page = pdf_file.getPage(page)
        pdf_page.mergeTranslatedPage(pdf_page.getContents().replace(b'hi', b'hello'), 0, 0)
        pdf_writer.addPage(pdf_page)
except Exception as e:
    print(f"Error: An error occurred while processing the PDF file: {e}")
    exit(1)

try:
    with open('new_resume.pdf', 'wb') as fh:
        pdf_writer.write(fh)
except Exception as e:
    print(f"Error: An error occurred while writing the new PDF file: {e}")
    exit(1)

print("Successfully replaced 'hi' with 'hello' in the PDF file.")

Error: An error occurred while processing the PDF file: 'EncodedStreamObject' object has no attribute 'replace'
Successfully replaced 'hi' with 'hello' in the PDF file.
